<h2>Ejercicio 2</h2>

Importar dataset

In [24]:
import pandas as pd

df = pd.read_csv("data/usuarios_sucios.csv")
df.shape

(1060, 11)

In [25]:
df.head()

id_usuario          nombre                   email fecha_registro  \
0         NaN  Camila Fuentes                     NaN     05-11-2023   
1         NaN   Javiera Silva         correo_invalido     2025-05-19   
2         3.0   Javiera Silva  javiera836@outlook.com            NaN   
3         4.0  camila fuentes   camila307@outlook.com            NaN   
4         5.0   JAVIERA SILVA                     NaN     06/05/2025   

        ciudad  plan         telefono utm_source utm_campaign  \
0   La Florida   NaN  +56 9 4517 1614        NaN      SUMMER    
1  Puente Alto  Free  +56 9 7499 1812     google  blackfriday   
2   La Florida   Pro            9-123     google       summer   
3  PUENTE ALTO  Free              NaN   linkedin      SUMMER    
4          NaN  Free            9-123   linkedin      SUMMER    

   columna_innecesaria1 columna_innecesaria2  
0                1145.0                    x  
1                   NaN                    y  
2                   NaN                       
3                4718.0                    x  
4                   NaN                    x

1.Eliminar columnas innecesarias.

In [26]:
df2 = df.drop(columns=["columna_innecesaria1","columna_innecesaria2"])
df2.head()

id_usuario          nombre                   email fecha_registro  \
0         NaN  Camila Fuentes                     NaN     05-11-2023   
1         NaN   Javiera Silva         correo_invalido     2025-05-19   
2         3.0   Javiera Silva  javiera836@outlook.com            NaN   
3         4.0  camila fuentes   camila307@outlook.com            NaN   
4         5.0   JAVIERA SILVA                     NaN     06/05/2025   

        ciudad  plan         telefono utm_source utm_campaign  
0   La Florida   NaN  +56 9 4517 1614        NaN      SUMMER   
1  Puente Alto  Free  +56 9 7499 1812     google  blackfriday  
2   La Florida   Pro            9-123     google       summer  
3  PUENTE ALTO  Free              NaN   linkedin      SUMMER   
4          NaN  Free            9-123   linkedin      SUMMER

2.Convertir “fecha de registro” a datetime.

In [27]:
s = df2["fecha_registro"].astype("string").str.strip()
s = s.str.replace(".", "/", regex=False).str.replace("-", "/", regex=False)

df2["fecha_registro_2"] = pd.to_datetime(
    s,
    errors="coerce"
)

In [28]:
# NaT (Not a Time)
df2.head()

id_usuario          nombre                   email fecha_registro  \
0         NaN  Camila Fuentes                     NaN     05-11-2023   
1         NaN   Javiera Silva         correo_invalido     2025-05-19   
2         3.0   Javiera Silva  javiera836@outlook.com            NaN   
3         4.0  camila fuentes   camila307@outlook.com            NaN   
4         5.0   JAVIERA SILVA                     NaN     06/05/2025   

        ciudad  plan         telefono utm_source utm_campaign fecha_registro_2  
0   La Florida   NaN  +56 9 4517 1614        NaN      SUMMER        2023-05-11  
1  Puente Alto  Free  +56 9 7499 1812     google  blackfriday              NaT  
2   La Florida   Pro            9-123     google       summer              NaT  
3  PUENTE ALTO  Free              NaN   linkedin      SUMMER               NaT  
4          NaN  Free            9-123   linkedin      SUMMER        2025-06-05

In [29]:
df2.dtypes

id_usuario                 float64
nombre                      object
email                       object
fecha_registro              object
ciudad                      object
plan                        object
telefono                    object
utm_source                  object
utm_campaign                object
fecha_registro_2    datetime64[ns]
dtype: object

3.Crear índice con “id_usuario”.

In [30]:
print(f"Tipo de índice actual: {type(df.index)}")
print(f"Nombre del índice: {df.index.name}")
print(f"Primeros valores del índice: {df.index[:5]}")

Tipo de índice actual: <class 'pandas.core.indexes.range.RangeIndex'>
Nombre del índice: None
Primeros valores del índice: RangeIndex(start=0, stop=5, step=1)


In [31]:
duplicados = df["id_usuario"].duplicated().sum()

In [32]:
if duplicados > 0:
    print(f"Atención: Tienes {duplicados} IDs repetidos. El índice no será único.")
else:
    print("Todos los IDs son únicos. ¡Perfecto para un índice!")

Atención: Tienes 386 IDs repetidos. El índice no será único.


In [33]:
# Establecer 'id_usuario' como el nuevo índice
df = df.set_index("id_usuario")

In [34]:
# 3. Comprobar el cambio
print(f"Nuevo tipo de índice: {type(df.index)}")
print(f"Nombre actual del índice: {df.index.name}")

Nuevo tipo de índice: <class 'pandas.core.indexes.base.Index'>
Nombre actual del índice: id_usuario


In [35]:
# Buscar toda la información del usuario con ID 500
datos_usuario = df.loc[500]
datos_usuario

nombre                       MATÍAS GONZÁLEZ
email                   matías269@empresa.cl
fecha_registro                    2023-01-22
ciudad                              Santiago
plan                              Enterprise
telefono                     +56 9 4916 1096
utm_source                          GOOGLE  
utm_campaign                     blackfriday
columna_innecesaria1                  6904.0
columna_innecesaria2                       x
Name: 500.0, dtype: object

4.Cambiar nombres de columnas con .rename()

In [36]:
df2.columns

Index(['id_usuario', 'nombre', 'email', 'fecha_registro', 'ciudad', 'plan',
       'telefono', 'utm_source', 'utm_campaign', 'fecha_registro_2'],
      dtype='object')

In [37]:
# Renombrar columnas específicas
df2 = df2.rename(columns={
    "fecha_registro": "fecha_original",
    "fecha_registro_2": "fecha_limpia",
    "utm_source": "fuente_marketing",
    "utm_campaign": "campana"
})
df2.columns

Index(['id_usuario', 'nombre', 'email', 'fecha_original', 'ciudad', 'plan',
       'telefono', 'fuente_marketing', 'campana', 'fecha_limpia'],
      dtype='object')

5.Eliminar duplicados según combinación de nombre y fecha.

In [39]:
# 1. Definir las columnas que forman la combinación única
columnas_clave = ["nombre", "fecha_limpia"]

# 2. Contar cuántos hay antes (opcional, para tu registro)
antes = len(df2)
antes

1060

In [41]:
# 3. Eliminar duplicados manteniendo solo la primera aparición
df3 = df2.drop_duplicates(subset=columnas_clave, keep="first")

# 4. Ver el resultado
despues = len(df3)
print(despues)
print(f"Se eliminaron {antes - despues} registros duplicados.")

178
Se eliminaron 882 registros duplicados.
